# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,293.02,100,75,10.29,PT,1700993491
1,1,andergrove,-21.0833,149.1833,298.51,85,100,3.08,AU,1700993492
2,2,waitangi,-43.9535,-176.5597,287.49,97,94,1.34,NZ,1700993492
3,3,longyearbyen,78.2186,15.6401,268.06,80,75,2.06,SJ,1700993492
4,4,puerto natales,-51.7236,-72.4875,277.24,83,78,6.40,CL,1700993492


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] - 273.15 < 27) & (city_data_df["Max Temp"] - 273.15 > 21) &
                                     (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
filtered_cities_df.dropna(how = "any")

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,tionk essil,12.7856,-16.5217,299.83,43,0,1.85,SN,1700993509
113,113,tura,25.5198,90.2201,297.35,61,0,1.18,IN,1700993519
257,257,vila velha,-20.3297,-40.2925,296.12,77,0,3.58,BR,1700993436
489,489,pasighat,28.0667,95.3333,296.47,53,0,0.20,IN,1700993622
503,503,bubaque,11.2833,-15.8333,300.03,59,0,3.07,GW,1700993625
509,509,neka,36.6508,53.2990,294.88,56,0,3.09,IR,1700993627
565,565,khark,29.2614,50.3306,298.16,61,0,3.09,IR,1700993641


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
75,tionk essil,SN,12.7856,-16.5217,43,
113,tura,IN,25.5198,90.2201,61,
257,vila velha,BR,-20.3297,-40.2925,77,
489,pasighat,IN,28.0667,95.3333,53,
503,bubaque,GW,11.2833,-15.8333,59,
509,neka,IR,36.6508,53.2990,56,
565,khark,IR,29.2614,50.3306,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tionk essil - nearest hotel: Campement Edonia
tura - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
pasighat - nearest hotel: Aane Hotel
bubaque - nearest hotel: Chez Julio
neka - nearest hotel: هتل میلاد نکا
khark - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
75,tionk essil,SN,12.7856,-16.5217,43,Campement Edonia
113,tura,IN,25.5198,90.2201,61,No hotel found
257,vila velha,BR,-20.3297,-40.2925,77,Hotel Prainha
489,pasighat,IN,28.0667,95.3333,53,Aane Hotel
503,bubaque,GW,11.2833,-15.8333,59,Chez Julio
509,neka,IR,36.6508,53.2990,56,هتل میلاد نکا
565,khark,IR,29.2614,50.3306,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)